In [1]:
# Cargar CIFAR-10 desde torchvision
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

# Descargar y cargar los datos
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Convertir a numpy
X_train = np.array([np.array(img[0]).transpose(1, 2, 0) for img in trainset])
y_train = np.array(trainset.targets)

X_test = np.array([np.array(img[0]).transpose(1, 2, 0) for img in testset])
y_test = np.array(testset.targets)

# Submuestreo para rapidez
X_train = X_train[:5000]
y_train = y_train[:5000]
X_test  = X_test[:500]
y_test  = y_test[:500]

# Aplanar imágenes
X_train = X_train.reshape(X_train.shape[0], -1)
X_test  = X_test.reshape(X_test.shape[0], -1)

# Normalizar datos
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test  -= mean_image


100%|██████████| 170M/170M [00:02<00:00, 76.3MB/s]


In [2]:
def svm_loss_naive(W, X, y, reg):
    dW = np.zeros_like(W)
    num_classes = W.shape[1]
    num_train = X.shape[0]
    loss = 0.0

    for i in range(num_train):
        scores = X[i].dot(W)
        correct_class_score = scores[y[i]]
        for j in range(num_classes):
            if j == y[i]:
                continue
            margin = scores[j] - correct_class_score + 1  # delta = 1
            if margin > 0:
                loss += margin
                dW[:, j] += X[i]
                dW[:, y[i]] -= X[i]

    loss /= num_train
    dW   /= num_train

    # Regularización
    loss += 0.5 * reg * np.sum(W * W)
    dW   += reg * W
    return loss, dW


In [3]:
# Inicializar pesos aleatoriamente
np.random.seed(0)
W = 0.001 * np.random.randn(X_train.shape[1], 10)

# Hiperparámetros
learning_rate = 1e-7
reg = 2.5e4
num_iters = 1500

# Entrenamiento con descenso de gradiente
for it in range(num_iters):
    loss, grad = svm_loss_naive(W, X_train, y_train, reg)
    W -= learning_rate * grad

    if it % 100 == 0:
        print(f'Iter {it}/{num_iters}: Loss {loss:.4f}')


Iter 0/1500: Loss 388.3619
Iter 100/1500: Loss 238.9495
Iter 200/1500: Loss 148.3829
Iter 300/1500: Loss 93.4860
Iter 400/1500: Loss 60.2101
Iter 500/1500: Loss 40.0399
Iter 600/1500: Loss 27.8137
Iter 700/1500: Loss 20.4028
Iter 800/1500: Loss 15.9106
Iter 900/1500: Loss 13.1877
Iter 1000/1500: Loss 11.5372
Iter 1100/1500: Loss 10.5368
Iter 1200/1500: Loss 9.9303
Iter 1300/1500: Loss 9.5628
Iter 1400/1500: Loss 9.3399


In [4]:
# Predicciones
y_pred = np.argmax(X_test.dot(W), axis=1)
accuracy = np.mean(y_pred == y_test)
print(f'Accuracy - precisionn en test: {accuracy:.4f}')


Accuracy en test: 0.2220
